In [1]:
import pyspark
from pyspark.sql import SparkSession



In [2]:
pyspark.__version__

'3.1.2'

In [3]:
#Setting Spark installation location to jupyter
import os
import sys

os.environ["SPARK_HOME"] = "/usr/local/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"

os.environ["PYSPARK_PYTHON"] = "/usr/local/spark-3.1.2-bin-hadoop3.2/python" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/local/spark-3.1.2-bin-hadoop3.2/python"

sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.9-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

In [4]:
!wget 'https://repository.cloudera.com/repository/cloudera-repos/org/apache/hive/hive-hbase-handler/2.1.1-cdh6.2.0/hive-hbase-handler-2.1.1-cdh6.2.0.jar' -O 'hive-hbase-handler-2.1.1-cdh6.2.0.jar'

!sudo cp ~/work/hive-hbase-handler-2.1.1-cdh6.2.0.jar /usr/local/spark/jars/hive-hbase-handler-2.1.1-cdh6.2.0.jar



--2024-04-30 23:22:22--  https://repository.cloudera.com/repository/cloudera-repos/org/apache/hive/hive-hbase-handler/2.1.1-cdh6.2.0/hive-hbase-handler-2.1.1-cdh6.2.0.jar
Resolving repository.cloudera.com (repository.cloudera.com)... 52.39.184.253, 44.229.98.58
Connecting to repository.cloudera.com (repository.cloudera.com)|52.39.184.253|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118972 (116K) [application/java-archive]
Saving to: ‘hive-hbase-handler-2.1.1-cdh6.2.0.jar’

hive-hbase-handler- 100%[===================>] 116.18K   533KB/s    in 0.2s    

2024-04-30 23:22:22 (533 KB/s) - ‘hive-hbase-handler-2.1.1-cdh6.2.0.jar’ saved [118972/118972]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Read HBase Table using PySpark Demo") \
    .config("spark.jars", "hive-hbase-handler-2.1.1-cdh6.2.0.jar") \
    .config('hive.metastore.uris', 'thrift://hive-metastore:9083')\
    .config("spark.executor.extraClassPath", "/usr/local/spark/jars/hive-hbase-handler-2.1.1-cdh6.2.0.jar") \
    .config("spark.executor.extraLibrary", "/usr/local/spark/jars/hive-hbase-handler-2.1.1-cdh6.2.0.jar") \
    .config("spark.driver.extraClassPath", "/usr/local/spark/jars/hive-hbase-handler-2.1.1-cdh6.2.0.jar") \
    .enableHiveSupport()\
    .getOrCreate()

24/04/30 23:22:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/30 23:22:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
print(spark.sparkContext.appName)

Read HBase Table using PySpark Demo


In [10]:
transaction_detail_df = spark.sql("Select * from default.student")
# transaction_detail_df = spark.sql("select info from student;")

AnalysisException: Table or view not found: default.student; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [default, student], [], false


In [34]:

# Define the HBase table name
hbase_table_name = "student"

# Read data from the HBase table into a DataFrame
df = spark.read.format("org.apache.hadoop.hbase.spark") \
    .option("hbase.table", hbase_table_name) \
    .option("hbase.columns.mapping", "info:name STRING, info:age STRING, info:gender STRING, info:grade STRING") \
    .load()

# Show the DataFrame
df.show()

Py4JJavaError: An error occurred while calling o155.load.
: java.lang.ClassNotFoundException: Failed to find data source: org.apache.hadoop.hbase.spark. Please find packages at http://spark.apache.org/third-party-projects.html
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:692)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:746)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:265)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:225)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j

In [26]:
spark.sparkContext.table("default.student").show()

AttributeError: 'SparkContext' object has no attribute 'table'

In [ ]:
from pyspark.sql import SparkSession

def main():
    spark = SparkSession.builder.master("local").appName("HelloSpark").getOrCreate()

    dataSourceFormat = "org.apache.spark.sql.execution.datasources.hbase"
    writeCatalog = ''.join("""{
                "table":{"namespace":"default", "name":"student", "tableCoder":"PrimitiveType"},
                "rowkey":"key",
                "columns":{
                  "key":{"cf":"rowkey", "col":"key", "type":"int"},
                  "empId":{"cf":"personal","col":"empId","type":"string"},
                  "empName":{"cf":"personal", "col":"empName", "type":"string"},
                  "empWeight":{"cf":"personal", "col":"empWeight", "type":"double"}
                }
              }""".split())

    writeDF = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/test/emp.csv")
    print("csv file read", writeDF.show())
    writeDF.write.options(catalog=writeCatalog, newtable=5).format(dataSourceFormat).save()
    print("csv file written to HBase")

    readCatalog = ''.join("""{
                "table":{"namespace":"default", "name":"tblEmployee"},
                "rowkey":"key",
                "columns":{
                  "key":{"cf":"rowkey", "col":"key", "type":"int"},
                  "empId":{"cf":"personal","col":"empId","type":"string"},
                  "empName":{"cf":"personal", "col":"empName", "type":"string"}
                }
              }""".split())

    print("going to read data from Hbase table")
    readDF = spark.read.options(catalog=readCatalog).format(dataSourceFormat).load()
    print("data read from HBase table")
    readDF.select("empId", "empName").show()
    readDF.show()

# entry point for PySpark application
if __name__ == '__main__':
    main()

In [22]:

transaction_detail_df = spark.sql("select * from student")

AnalysisException: Table or view not found: student; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [student], [], false


In [20]:
transaction_detail_df.show(5)

++
||
++
||
++



In [10]:
!pip uninstall pyspark

Found existing installation: pyspark 3.1.2
Can't uninstall 'pyspark'. No files were found to uninstall.


In [3]:
!pip uninstall pyspark

Found existing installation: pyspark 3.1.2
Can't uninstall 'pyspark'. No files were found to uninstall.


In [8]:
!pip install pyspark==3.1.0

ERROR: Could not find a version that satisfies the requirement pyspark==3.1.0 (from versions: 2.1.2, 2.1.3, 2.2.0.post0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.4.5, 2.4.6, 2.4.7, 2.4.8, 3.0.0, 3.0.1, 3.0.2, 3.0.3, 3.1.1, 3.1.2, 3.1.3, 3.2.0, 3.2.1, 3.2.2, 3.2.3, 3.2.4, 3.3.0, 3.3.1, 3.3.2, 3.3.3, 3.3.4, 3.4.0, 3.4.1, 3.4.2, 3.4.3, 3.5.0, 3.5.1)
ERROR: No matching distribution found for pyspark==3.1.0


In [5]:
print(pyspark.__version__)

3.1.2


In [2]:
!pip install delta

In [7]:
import delta

dir(delta)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'parse']

In [ ]:
# Create a Spark DataFrame
data = spark.range(0, 5)

# Write to a Delta Lake table
(data
   .write
   .format("delta")
   .save("/tmp/delta-table")
)

# Read from the Delta Lake table
df = (spark
        .read
        .format("delta")
        .load("/tmp/delta-table")
        .orderBy("id")
      )

# Show the Delta Lake table
df.show()

In [ ]:
# Create a Spark DataFrame
data = spark.range(0, 5)

# Write to a Delta Lake table
(data
   .write
   .format("delta")
   .save("/tmp/delta-table")
)

# Read from the Delta Lake table
df = (spark
        .read
        .format("delta")
        .load("/tmp/delta-table")
        .orderBy("id")
      )

# Show the Delta Lake table
df.show()

---

In [8]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("key", IntegerType(), True),
    StructField("value", StringType(), True)
])

hbase_storage_handler = org.apache.hadoop.hive.hbase.HBaseStorageHandler(schema)

NameError: name 'org' is not defined